### create data from stock

In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from curl_cffi import requests

SYMBOL = "IFX.DE"
HISTORY = "10y"

class stock_datasets:
    CLASSES = ["Bull", "Bear"]
    def __init__(self, stock=None, period=None, interval="1d"):
        self._stock = stock if stock is not None else None
        self._period = period if period is not None else None
        self._interval = interval if interval is not None else None
        self._stock_data = None
        self._X_train = None
        self._Y_train = None
        self._X_test = None
        self._Y_test = None

    @property
    def stock_data(self):
        return self._stock_data

    @property
    def features(self):
        return self._features
        
    def load_stock(self, stock=None, period=None, interval=None):
        ## read stock data from stock markt
        self._stock = stock if stock is not None else self._stock
        self._period = period if period is not None else self._period
        self._interval = interval if interval is not None else self._interval    
        self.session = requests.Session(impersonate="chrome")
        self._stock_data = yf.Ticker(self._stock, session=self.session).history(period=self._period, interval=self._interval)
        ### default all features
        self._base_features = ['Open', 'High', 'Low', 'Close', 'Volume', "Dividends", "Stock Splits"]
        self._features = None
        self._data = self._stock_data ## used for prediction process
        
    def set_working_data(self, features=(0, 5), remove_col=["Dividends", "Stock Splits"]):
        self._features = self._data[[self._base_features[f] for f in range(features[0],features[1])]]
        self._remove_columns(remove_col)
        self._normalize_data()
    
    ### remove meaningless columns
    def _remove_columns(self, columns):
        if len(columns) == 0:
            return
        self._data = self._stock_data.drop(columns=columns)
        self._data = self._stock_data[:-1]

    def _normalize_data(self):
        scaler = MinMaxScaler(feature_range=(0, 1))
        self._data = scaler.fit_transform(self._data)

    def _create_sequences(self, window_size):
        data_num = len(self._data)
        if data_num < window_size:
            raise ValueError("Too few number of data")
        X, Y = [], []
        for i in range(data_num - window_size):
            # Input: add all features: Open, High, Low, Close, Volume
            X.append(self._data[i:i+window_size, :]) 
            # Output: predict Close price
            Y.append(self._data[i+window_size], 3) 
        return np.array(X), np.array(Y)

    def create_train_test_data(self, window_size=100, train_data_percentage=0.7):
        if data_num < window_size:
            raise ValueError("Too few number of data")
        self.window_size = window_size
        if train_data_percentage >= 1.0:
            raise ValueError("Percent must less than 1.0")
        X, Y = self._create_sequences(window_size)
        split = int(train_data_percentage * len(X))
        self._X_train, self._X_test = X[:split], X[split:]
        self._Y_train, self._Y_test = Y[:split], Y[split:]

    def create_LSTM_model(self):
        model = Sequential([
            LSTM(64, return_sequences=True, input_shape=(self.window_size, self._features.shape[2])),  # 输入维度为 (窗口大小, 特征数)
            Dropout(0.3),
            LSTM(32, return_sequences=False),
            Dropout(0.2),
            Dense(1)
        ])


In [4]:
stock_daten = stock_datasets(SYMBOL, HISTORY)
stock_daten.load_stock()

In [5]:
stock_daten.set_working_data()

In [6]:
stock_daten.stock_data.tail()

AttributeError: 'numpy.ndarray' object has no attribute 'tail'

#### construct datasets

In [7]:
stock_daten.features.shape

(2543, 5)